In [8]:
from dotenv import load_dotenv
from huggingface_hub import login
import os
import pandas as pd

load_dotenv()
hftoken = os.getenv("HUGGINGFACE_TOKEN")
login(hftoken)

In [5]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

max_seq_length = 2048
dtype = None # None으로 지정할 경우 해당 컴퓨팅 유닛에 알맞은 dtype으로 저장됩니다. Tesla T4와 V100의 경우에는 Float16, Ampere+ 이상의 경우에는 Bfloat16으로 설정됩니다.
load_in_4bit = True # 메모리 사용량을 줄이기 위해서는 4bit 양자화를 사용하실 것을 권장합니다.

# 모델 및 토크나이저 선언
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # gated model을 사용할 경우 허깅페이스 토큰을 입력해주시길 바라겠습니다.
)
# LoRA Adapter 선언
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # 0을 넘는 숫자를 선택하세요. 8, 16, 32, 64, 128이 추천됩니다.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], # target module도 적절하게 조정할 수 있습니다.
    lora_alpha = 16,
    lora_dropout = 0, # 어떤 값이든 사용될 수 있지만, 0으로 최적화되어 있습니다.
    bias = "none",    # 어떤 값이든 사용될 수 있지만, "none"으로 최적화되어 있습니다.
    use_gradient_checkpointing = "unsloth", # 매우 긴 context에 대해 True 또는 "unsloth"를 사용하십시오.
    random_state = 42,
    use_rslora = False,
    loftq_config = None
)
prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
아래는 지시사항과 추가 정보가 포함된 내용입니다. 주어진 내용을 참고하여 한국어로 적절하게 답변해 주세요.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

==((====))==  Unsloth 2024.10.3: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe MIG 3g.40gb. Max memory: 39.5 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [6]:
def formatting_prompts_func1(examples):
    instructions = examples["question"]
    outputs = examples["response"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return { "formatted_text" : texts, }
pass

def formatting_prompts_func2(examples):
    instructions = examples["text"]
    outputs = examples["text"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return { "formatted_text" : texts, }
pass

dataset1 = load_dataset("LDC-ai/dataset", split='train')
dataset1 = dataset1.map(formatting_prompts_func1, batched = True,)

dataset2 = load_dataset("Linq-AI-Research/FinanceRAG",'FinQABench', split = "corpus")
dataset2 = dataset2.map(formatting_prompts_func2, batched = True,)

# dataset3 = load_dataset("BAAI/IndustryCorpus_finance", split = "train")
# dataset3 = dataset3.map(formatting_prompts_func2, batched = True,)

# 데이터셋 통합
combined_dataset = concatenate_datasets([dataset1, dataset2])

combined_dataset

Generating train split:   0%|          | 0/21950 [00:00<?, ? examples/s]

Map:   0%|          | 0/21950 [00:00<?, ? examples/s]

Generating corpus split:   0%|          | 0/92 [00:00<?, ? examples/s]

Generating queries split:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

Dataset({
    features: ['', 'question', 'response', 'formatted_text', '_id', 'title', 'text'],
    num_rows: 22042
})

In [12]:
df = pd.DataFrame(combined_dataset)

df['formatted_text'][0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n아래는 지시사항과 추가 정보가 포함된 내용입니다. 주어진 내용을 참고하여 한국어로 적절하게 답변해 주세요.\n\n### Instruction:\n비배당 주식에 대한 미국형 콜 옵션의 가격을 정량적으로 계산하는 과정에 대해 설명하시오. 특히, 다음의 두 가지 식을 비교하면서 논의하시오. 유럽형 콜 옵션의 가격은 다음과 같이 주어집니다: \n\n\n### Response:\n비배당 주식에 대한 미국형 콜 옵션의 가격을 정량적으로 계산하려면 여러 방법이 있습니다. 그러나 일반적으로 가장 널리 사용되는 방법은 블랙-숄즈 모델(Black-Scholes Model)입니다. 그러나 미국형 옵션은 조기 행사 가능성이 있기 때문에 유럽형 옵션의 가격을 기반으로 하는 블랙-숄즈<|im_end|>'

In [18]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # True로 설정하면 짧은 텍스트 데이터에 대해서는 더 빠른 학습 속도로를 보여줍니다.
    args = TrainingArguments( # TrainingArguments는 자신의 학습 환경과 기호에 따라 적절하게 설정하면 됩니다.
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt_format.format(
        """다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.
### 질문: 다음 중 우리나라 주식시장 매매 제도에 대한 기술로 맞는 것은?
### 선택지: 
A. 개장 시간은 오전 10시다.
B. 유가증권시장의 가격 제한폭은 전일 종가 대비 상하 15%이다.
C. 코스닥시장에는 가격 제한폭이 없다.
D. 점심시간(12~1시)에는 휴장한다.
E. 동시호가는 폐장 시간에만 적용한다.
F. K-OTC시장의 가격 제한폭은 전일 종가 대비 상하 30%이다.
G. K-OTC시장의 운영시간은 09:00부터 16:00까지이다.""", # instruction
        "", # output 생성을 위해 빈 칸으로 설정
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
print(tokenizer.batch_decode(outputs))
for item in tokenizer.batch_decode(outputs):
    print(item)

max_steps is given, it will override any value given in num_train_epochs


['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n아래는 지시사항과 추가 정보가 포함된 내용입니다. 주어진 내용을 참고하여 한국어로 적절하게 답변해 주세요.\n\n### Instruction:\n다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문: 다음 중 우리나라 주식시장 매매 제도에 대한 기술로 맞는 것은?\n### 선택지: \nA. 개장 시간은 오전 10시다.\nB. 유가증권시장의 가격 제한폭은 전일 종가 대비 상하 15%이다.\nC. 코스닥시장에는 가격 제한폭이 없다.\nD. 점심시간(12~1시)에는 휴장한다.\nE. 동시호가는 폐장 시간에만 적용한다.\nF. K-OTC시장의 가격 제한폭은 전일 종가 대비 상하 30%이다.\nG. K-OTC시장의 운영시간은 09:00부터 16:00까지이다.\n\n### Response:\n본 질문의 정답을 찾기 위해 각 선택지들을 살펴봅니다.\n\nA. 개장 시간은 오전 10시다.\n이 문장은 주식 시장의 개장 시간에 대한 정보이며, 일반적으로 주식 시장의 개장 시간은 이와 같으므로 올바른 정보라고 판단됩니다.\n\nB. 유가증권시장의 가격 제한폭은 전일 종가 대비 상하 15%이다.\n이 문장은 주식 시장의 가격 제한폭에 대한 정보이며, 유가증권시장의 가격 제한폭이 전일 종가 대비 상하 15%라는 정보는 일반적으로 주식 시장의 규칙에 따른 정보라고 판단됩니다.\n\nC. 코스닥시장에는 가격 제한폭이 없다.\n코스닥시장에 대한 정보입니다. 실제로 코스닥시장에는 가격 제한폭이 있으므로 이 문장은 잘못된 정보라고 판단됩니다.\n\nD. 점심시간(12~1시)에는 휴장한다.\n이 문장은 주식 시장의 휴장 시간에 대한 정보이며, 대부분의 주식 시장에서 점심시간에는 휴장하지 않는다

In [11]:
# # LoRA Adapter 저장
# model.save_pretrained("lora_model")
# tokenizer.save_pretrained("lora_model")

# # Merged model 저장 및 업로드
# model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("LDC-ai/test_model", tokenizer, save_method = "merged_16bit", token = hftoken) # 개인 huggingface token을 사용하여 업로드할 수 있습니다.

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 29.99 out of 48.0 RAM for saving.


100%|██████████| 28/28 [00:00<00:00, 29.74it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: You are pushing to hub, but you passed your HF username = LDC-ai.
We shall truncate LDC-ai/test_model to test_model


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 29.94 out of 48.0 RAM for saving.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/LDC-ai/test_model
